In [35]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import math
from math import radians, cos, sin, asin, sqrt
import sys
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier

In [36]:
# 左下角经纬度坐标
lb_lon = 121.20120490000001
lb_lat = 31.28175691

# 上角经纬度坐标
rt_lon = 121.2183295
rt_lat = 31.29339344

# 栅格边长（左下角栅格坐标为 (0, 0) ）
grid_len = 20

In [37]:
# 根据两点经纬度计算两点间距离
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000  

In [38]:
# 横向最大距离
max_distance_x =  haversine(lb_lon, lb_lat, rt_lon, lb_lat)
# 右上角栅格横坐标
int(max_distance_x / 20)

81

In [39]:
# 纵向最大距离
max_distance_y = haversine(lb_lon, lb_lat, lb_lon, rt_lat)
# 右上角栅格纵坐标
int(max_distance_y / 20)

64

In [40]:
# 将地图栅格化，将经纬度坐标转化为栅格坐标，左下角的栅格坐标是 (0, 0) ，右上角的栅格坐标是 (81, 64)
# 再将栅格坐标转化为栅格ID（0 ～ 82 * 65 - 1 = 5329）共 5330 个栅格，左下角的栅格ID是 0 ，右上角的栅格ID是 5329
def ll_to_gridID(lon, lat):
    # 左下角经纬度坐标，栅格边长
    global lb_lon, lb_lat, grid_len
    
    # 将经纬度坐标转化为栅格坐标
    X = int(haversine(lon, lb_lat, lb_lon, lb_lat) / grid_len) 
    Y = int(haversine(lb_lon, lat, lb_lon, lb_lat) / grid_len)
    
    # 将栅格坐标转化为栅格ID：gridID = X + Y * 82
    gridID = X + Y * 82
    
    return (gridID)

In [41]:
# 将栅格ID转化为栅格中心经纬度
def gridID_to_ll(gridID):
    # 左下角经纬度坐标，右上角经纬度坐标，栅格边长
    global lb_lon, lb_lat, rt_lon, rt_lat
    
    # 将栅格ID转化为栅格坐标
    X = gridID % 82
    Y = int(gridID / 82)
    
    # 将栅格坐标转化为经纬度坐标
    # 栅格的边和栅格的中心位置将横向长度划分为 82 * 2 = 164 份
    # 栅格的边和栅格的中心位置将纵向长度划分为 65 * 2 = 130 份
    # 每一份的经度差：
    delta_lon = (rt_lon - lb_lon) / 164
    # 每一份的纬度差：
    delta_lat = (rt_lat - lb_lat) / 130
    # 栅格坐标横坐标为X的栅格的中心点经度：
    lon = lb_lon + (1 + 2 * X) * delta_lon
    # 栅格坐标纵坐标为Y的栅格的中心点经度：
    lat = lb_lat + (1 + 2 * Y) * delta_lat
    
    return [lon, lat]

In [42]:
# 合并两张表，用 data_2g 中的 RNCID_1，CellID_1 与 gongcan_2g 的 RNCID，CellID 匹配，将基站的经纬度信息加到 data_2g 中
def merge_data_gongcan():
    data_2g = pd.read_csv('../raw_data/data_2g.csv')
    gongcan_2g = pd.read_csv('../raw_data/2g_gongcan.csv')
    
    for i in range(1, 8):
        # 换掉 gongcan_2g 的列名用以和 data_2g merge
        gongcan_2g.columns = ['RNCID_' + str(i), 'CellID_' + str(i), 'Lat_' + str(i), 'Lon_' + str(i)]    
        data_2g = pd.merge(data_2g, gongcan_2g, how='left', on=['RNCID_' + str(i), 'CellID_' + str(i)])
        
    # 将 RSSI_1 ~ RSSI_7 的空缺值nan用 -sys.maxsize - 1 来代替
    for j in range(1, 8):
        data_2g['RSSI_' + str(j)] = data_2g['RSSI_' + str(j)].fillna(-sys.maxsize - 1)
    
    # 将其余空缺值nan替换为-1
    data_2g = data_2g.fillna(-1)
    return data_2g

In [43]:
# 根据MR数据的GPS加上栅格ID
def add_gridID(data):
    data['GridID'] = data.apply(lambda x: ll_to_gridID(x.Longitude, x.Latitude), axis = 1)
    return data

In [44]:
# 随机选取80%的数据记录作为训练集，余下20%作为测试集合
def data_train_test_split(data, random_seed):
    # 样本特征集（将 DataFrame 转化为 list）
    X = data[['Lon_1', 'Lat_1', 'RSSI_1', 
              'Lon_2', 'Lat_2', 'RSSI_2', 
              'Lon_3', 'Lat_3', 'RSSI_3', 
              'Lon_4', 'Lat_4', 'RSSI_4',
              'Lon_5', 'Lat_5', 'RSSI_5',
              'Lon_6', 'Lat_6', 'RSSI_6',
              'Lon_7', 'Lat_7', 'RSSI_7',
             ]].values
    # 样本结果（将 DataFrame 转化为 list ）
    y = data['GridID'].values
    
    # 随机选取80%的数据记录作为训练集，余下20%作为测试集合
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
    
    return X_train, X_test, y_train, y_test

In [45]:
# 7个分类器
classifier_models = {
    # 高斯朴素贝叶斯分类器 Gaussian Naive Bayes (GaussianNB)
    'GaussianNB': GaussianNB(),
    # K近邻分类器 KNeighborsClassifier
    'KNeighbors': KNeighborsClassifier(n_neighbors=2),
    # 决策树分类器 DecisionTreeClassifier
    'DecisionTree': tree.DecisionTreeClassifier(criterion='entropy', max_depth=50),
    # 随机森林 RandomForestClassifier
    'RandomForest': RandomForestClassifier(max_depth=3, n_estimators=45),
    # AdaBoost AdaBoostClassifier
    'AdaBoost': AdaBoostClassifier(n_estimators=150, learning_rate = 0.1),
    # Bagging meta-estimator（Bagging 元估计器）BaggingClassifier
    'Bagging': BaggingClassifier(n_estimators=40),
    # GBDT（梯度树提升）GradientBoostingClassifier
    'GradientBoosting': GradientBoostingClassifier(n_estimators=40, max_depth=10)
}

In [46]:
# 传入不同的分类器，训练并且作出预测
def train_classifier_and_predict(classifier_model, X_train, X_test, y_train):
    clf = classifier_model
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    return y_pred

In [47]:
# 计算预测位置和证实位置的误差（采用欧式距离），按照计算误差从小到大进行排序
# 返回小到大进行排序的误差
def calculate_error_and_sort(y_pred, y_test):
    
    # 将预测结果里的栅格ID转化为经纬度坐标
    y_pred = list(map(gridID_to_ll, y_pred))
    # 将测试结果里的栅格ID转化为经纬度坐标
    y_test = list(map(gridID_to_ll, y_test))
    
    # 将预测结果和测试结果的经纬度坐标放在同一个list里，形成一个二维数组
    temp = [y_pred[i] + y_test[i] for i in range(min(len(y_pred),len(y_test)))]
    
    # 计算各个误差
    errors = [haversine(x[0], x[1], x[2], x[3]) for x in temp]
    # 按照计算误差从小到大进行排序
    sorted_errors = sorted(errors)
    
    return sorted_errors

In [51]:
# 计算10次误差的平均值（即平均误差）
def calculate_avg_errors(errors_list):
    errors_avg = []
    for i in range(len(errors_list[0])):
        errors_avg.append(np.mean([x[i] for x in errors_list]))
    return  errors_avg

In [58]:
# 调用前面的函数，传入分类器，返回10次预测结果和平均误差
def get_result(classifier_model, data):
    
    # 用以存储10次预测结果
    y_pred_list = []
    # 用以存储10次误差
    errors_list = []
    
    for i in range(0, 10):
        # 随机选取80%的数据记录作为训练集，余下20%作为测试集合，传入i作为random_seed
        X_train, X_test, y_train, y_test = data_train_test_split(data, i)
        # 训练并且作出预测
        y_pred = train_classifier_and_predict(classifier_model, X_train, X_test, y_train)
        # 存储本次预测结果
        y_pred_list.append(y_pred)
        # 计算预测位置和证实位置的误差（采用欧式距离），按照计算误差从小到大进行排序
        errors = calculate_error_and_sort(y_pred, y_test)
        # 存储本次误差排序
        errors_list.append(errors)
        
    # 计算10次误差的平均值
    avg_errors = calculate_avg_errors(errors_list)
    
    # 返回10次预测结果和平均误差
    return y_pred_list, avg_errors

In [59]:
def main():
    # 合并两张表
    data = merge_data_gongcan()
    
    # 根据MR数据的GPS换算成栅格ID作为结果
    data = add_gridID(data)
    
    y_pred_list, avg_errors = get_result(classifier_models['GaussianNB'], data)

    return y_pred_list, avg_errors

In [60]:
y_pred_list, avg_errors = main()

In [61]:
y_pred_list

[array([2006, 2006, 2006, ..., 4387, 4387, 2006]),
 array([2006, 2517, 3161, ..., 2517, 2517, 2006]),
 array([  54, 2270, 4164, ..., 2270, 3663, 3161]),
 array([3161, 3161, 4387, ..., 4387, 4387, 3161]),
 array([4387, 2006, 4387, ..., 3161, 2006, 4387]),
 array([  55, 4387, 4387, ..., 4164,   55, 4387]),
 array([2006, 5208, 2006, ..., 2006, 5208, 5208]),
 array([3161, 2270, 2270, ...,   55,   55, 2270]),
 array([2270,   54,   54, ..., 2270,   54,   54]),
 array([2270,   54, 4325, ..., 2270, 2270,   54])]

In [62]:
avg_errors

[0.0,
 1.9844226401851326,
 1.9844678579544741,
 7.9377320064594468,
 13.891007458970828,
 15.888018543067819,
 15.894295811787782,
 17.884837455404522,
 20.695606235421938,
 20.695704210781493,
 21.522215050258914,
 21.522305047917122,
 21.528516715982782,
 22.354828505906269,
 24.339109074529102,
 25.515662060989008,
 25.528148333972613,
 28.086053103579104,
 28.094511883891652,
 30.765261197561994,
 32.259840795814696,
 33.433029721475549,
 33.783686925757152,
 33.78389064451251,
 35.412741232618302,
 36.185157663595533,
 36.206059026252127,
 36.206092645410628,
 36.664903777660086,
 40.494406616371087,
 40.529683345822733,
 41.406657026032789,
 42.260895825826609,
 43.418872784387503,
 46.328504568575056,
 46.633716421985113,
 47.748631309794398,
 48.938316076070123,
 48.938589079946823,
 50.112605520222544,
 51.137621037479661,
 52.050282782074348,
 52.898795377793547,
 54.40865719902682,
 54.635045254128293,
 57.890864758902431,
 59.680236011890045,
 60.847042616565261,
 61.86619